In [5]:
# # !pip install awswrangler

# !pip uninstall -y numpy

# !pip uninstall -y setuptools



# !pip install setuptools

# !pip install numpy

In [1]:
import pandas as pd

import boto3
import numpy as np
import io
# import awswrangler as wr
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import s3_input, Session
import pickle
from sagemaker.amazon.amazon_estimator import get_image_uri

from sklearn.preprocessing import OneHotEncoder
# Save the encoded categorical model
import tempfile
import boto3
import joblib

import s3fs
from sagemaker.predictor import csv_serializer
import tempfile
import joblib
from sagemaker.predictor import csv_serializer

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append('../support_files/')

import constants as params

In [52]:
sess = sagemaker.Session()
role = get_execution_role()

bucket = "humana-data"
prefix_rawdata = 'rawdata/original_pq_files'
prefix_fe_data = "intermediate/data"
prefix_metadata = "intermediate/metadata"


conn = boto3.client('s3')
# contents = conn.list_objects(Bucket=bucket, Prefix=prefix)['Contents']

In [33]:
file_credit = "Credit.pq"

buffer = io.BytesIO()
s3 = boto3.resource('s3')
object_ = s3.Object('humana-data', '{}/{}'.format(prefix_rawdata, file_credit))
object_.download_fileobj(buffer)

credit_df = pd.read_parquet(buffer)
credit_df = credit_df.set_index("person_id_syn")
credit_df.columns = credit_df.columns.str.lower()

In [39]:
filename_metadata = prefix_metadata + "/" + "metadata"

# # to read the file
metadata_obj = conn.get_object(Bucket=bucket, Key=filename_metadata)
serializedObject = metadata_obj['Body'].read()
metadata = pickle.loads(serializedObject)

In [42]:
def credit_attributes(df, metadata):
    
    data_types = {}
    data_types['numeric_cols'] = ['credit_bal_1stmtgcredit_60dpd', 'credit_bal_agencyfirstmtg_60dpd', 
             'credit_bal_heloc_60dpd', 'credit_bal_nonagnfirstmtg_60dpd', 'credit_bal_nonmtgcredit_60dpd',
             'credit_bal_studentloan_60dpd', 'credit_bal_totalallcredit_60dpd', 'credit_bal_autobank',
             'credit_bal_autofinance', 'credit_bal_consumerfinance', 'credit_minmob_mtgcredit']
    
    data_types['categorical'] = []
    
    # data Transformation
    df['credit_num_new_accounts'] = df[['credit_num_autobank_new', 
                                                      'credit_num_autofinance_new',
                                                      'credit_num_consumerfinance_new',
                                                      'credit_num_mtgcredit_new']].astype(float).sum(axis=1)
    
    df['credit_num_collections'] = (df[['credit_num_mtg_collections',
                                                      'credit_num_totalallcredit_collections']].astype(float).sum(axis=1))/2

    df = df[data_types['numeric_cols'] + data_types['categorical']]
    
#     numeric_cols = data_types['numeric_cols'] + ['credit_num_new_accounts'] + ['credit_num_collections']


    metadata['credit'] = {}
    metadata['credit']['numeric'] = data_types['numeric_cols'].copy()
    
    return df, metadata

In [55]:
credit_fe_df.

,credit_bal_1stmtgcredit_60dpd,credit_bal_agencyfirstmtg_60dpd,credit_bal_heloc_60dpd,credit_bal_nonagnfirstmtg_60dpd,credit_bal_nonmtgcredit_60dpd,credit_bal_studentloan_60dpd,credit_bal_totalallcredit_60dpd,credit_bal_autobank,credit_bal_autofinance,credit_bal_consumerfinance,credit_minmob_mtgcredit
person_id_syn,,,,,,,,,,,
0002MOb79ST17bLYAe46eIc2,864.263515,364.200415,9.100900e+00,569.012631,3059.660249,3215.762022,5817.811692,2245.510843,2172.577397,699.143043,16.518833
0004cMOS6bTLf34Y7AIca8f3,1497.425917,85.131254,5.347506e+00,393.213767,2212.918611,1425.075480,3929.816290,4759.465844,3078.235986,866.308760,27.504949
000536M9O3ST98LaYaeA29Ia,741.709853,272.287671,8.514400e+01,798.359409,2541.222320,1777.517579,3302.597389,6064.546605,4606.637501,629.762015,27.843924
0009bMO9SfTLYe77A51I4ac3,907.747763,202.232738,3.000000e-01,552.034422,4129.590329,541.521616,4629.813478,6106.353309,6498.695809,1360.857637,23.165714
000M7OeS66bTL8bY89Aa16Ie,939.246970,254.559068,2.660359e+01,551.657225,3264.827959,2202.983742,4920.391294,2944.040395,5812.127058,1730.444909,18.201797
...,...,...,...,...,...,...,...,...,...,...,...
ffe33MOS25dTf027LaY7A5I3,964.755806,131.399457,2.330620e+01,638.056357,2075.081319,1086.997362,2059.514361,3311.871093,4322.692289,446.164158,27.555176
fff1M4O1cfST49LY464A2Ieb,1411.853429,273.292976,1.989153e+02,1058.378615,3163.041663,2935.336780,4990.222496,3545.681376,6198.366530,645.351258,18.909787
fff5MO7e401STLYcAd8e581I,1174.942615,556.955652,1.359666e+01,600.943906,1580.827142,1397.072181,3240.422600,2918.979683,7609.989205,1035.095176,28.049650


In [49]:
credit_fe_df, metadata = credit_attributes(df=credit_df, metadata = metadata)

file_credit_de ='credit_fe.parquet.gzip'
credit_fe_df.to_parquet(path = f's3://{bucket}/{prefix_fe_data}/{file_credit_fe}', compression='gzip', index=True)

conn.put_object(Bucket=bucket, Key=filename_metadata, Body=pickle.dumps(metadata))


{'ResponseMetadata': {'RequestId': '594D43D7WPK7WC55',
  'HostId': 'CX7VUcDcUJo5vdDW+Bj7bwAKWf56W1cUkV7FtKB4JMdRcMK2tVCPrPBo9RI7RfPFNcE7pPIziJs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'CX7VUcDcUJo5vdDW+Bj7bwAKWf56W1cUkV7FtKB4JMdRcMK2tVCPrPBo9RI7RfPFNcE7pPIziJs=',
   'x-amz-request-id': '594D43D7WPK7WC55',
   'date': 'Mon, 14 Nov 2022 20:41:34 GMT',
   'x-amz-version-id': '5oqvrMBcCUE9SiVq38JFG7SsevQDikJO',
   'etag': '"9e821adffaed7e9ee4de9652407d5894"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9e821adffaed7e9ee4de9652407d5894"',
 'VersionId': '5oqvrMBcCUE9SiVq38JFG7SsevQDikJO'}

In [57]:
file_credit_de ='credit_fe.parquet.gzip'

buffer = io.BytesIO()
s3 = boto3.resource('s3')
object_ = s3.Object('humana-data', '{}/{}'.format(prefix_fe_data, file_credit_de))
object_.download_fileobj(buffer)

credit_df = pd.read_parquet(buffer)

In [58]:
credit_df

,credit_bal_1stmtgcredit_60dpd,credit_bal_agencyfirstmtg_60dpd,credit_bal_heloc_60dpd,credit_bal_nonagnfirstmtg_60dpd,credit_bal_nonmtgcredit_60dpd,credit_bal_studentloan_60dpd,credit_bal_totalallcredit_60dpd,credit_bal_autobank,credit_bal_autofinance,credit_bal_consumerfinance,credit_minmob_mtgcredit
person_id_syn,,,,,,,,,,,
0002MOb79ST17bLYAe46eIc2,864.263515,364.200415,9.100900e+00,569.012631,3059.660249,3215.762022,5817.811692,2245.510843,2172.577397,699.143043,16.518833
0004cMOS6bTLf34Y7AIca8f3,1497.425917,85.131254,5.347506e+00,393.213767,2212.918611,1425.075480,3929.816290,4759.465844,3078.235986,866.308760,27.504949
000536M9O3ST98LaYaeA29Ia,741.709853,272.287671,8.514400e+01,798.359409,2541.222320,1777.517579,3302.597389,6064.546605,4606.637501,629.762015,27.843924
0009bMO9SfTLYe77A51I4ac3,907.747763,202.232738,3.000000e-01,552.034422,4129.590329,541.521616,4629.813478,6106.353309,6498.695809,1360.857637,23.165714
000M7OeS66bTL8bY89Aa16Ie,939.246970,254.559068,2.660359e+01,551.657225,3264.827959,2202.983742,4920.391294,2944.040395,5812.127058,1730.444909,18.201797
...,...,...,...,...,...,...,...,...,...,...,...
ffe33MOS25dTf027LaY7A5I3,964.755806,131.399457,2.330620e+01,638.056357,2075.081319,1086.997362,2059.514361,3311.871093,4322.692289,446.164158,27.555176
fff1M4O1cfST49LY464A2Ieb,1411.853429,273.292976,1.989153e+02,1058.378615,3163.041663,2935.336780,4990.222496,3545.681376,6198.366530,645.351258,18.909787
fff5MO7e401STLYcAd8e581I,1174.942615,556.955652,1.359666e+01,600.943906,1580.827142,1397.072181,3240.422600,2918.979683,7609.989205,1035.095176,28.049650
